## <span style="color:aquamarine">**Ensemble Learning: Bagging Tutorial**</span>

**We will use pima indian diabetes dataset to predict if a person has a diabetes or not based on certain features such as blood pressure, skin thickness, age etc. We will train a standalone model first and then use bagging ensemble technique to check how it can improve the performance of the model**

In [1]:
import warnings
warnings.filterwarnings("ignore")


import pandas as pd
df=pd.read_csv(r"F:\Machine Learning all Algorithms\18 Bagging and Boosting\pima-indians-diabetes.csv")
df

,6,148,72,35,0,33.6,0.627,50,1
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0
...,...,...,...,...,...,...,...,...,...
762,10,101,76,48,180,32.9,0.171,63,0
763,2,122,70,27,0,36.8,0.340,27,0
764,5,121,72,23,112,26.2,0.245,30,0
765,1,126,60,0,0,30.1,0.349,47,1


In [2]:
df.columns=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,85,66,29,0,26.6,0.351,31,0
1,8,183,64,0,0,23.3,0.672,32,1
2,1,89,66,23,94,28.1,0.167,21,0
3,0,137,40,35,168,43.1,2.288,33,1
4,5,116,74,0,0,25.6,0.201,30,0


In [3]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [4]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000,767.000000
mean,3.842243,120.859192,69.101695,20.517601,79.903520,31.990482,0.471674,33.219035,0.348110
std,3.370877,31.978468,19.368155,15.954059,115.283105,7.889091,0.331497,11.752296,0.476682
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243500,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,32.000000,32.000000,0.371000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,32.000000,127.500000,36.600000,0.625000,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [5]:
df.Outcome.value_counts()

Outcome
0    500
1    267
Name: count, dtype: int64

#### **There is slight imbalance in our dataset but since it is not major we will look into it later!**

### **Train Test Split**

In [6]:
X=df.drop('Outcome', axis=1)
y=df['Outcome']

In [7]:
# Scaling the data for better processing
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)
X_scaled[:3]

array([[-0.84372629, -1.12208597, -0.16024856,  0.53202348, -0.69355921,
        -0.68372895, -0.36426474, -0.18894038],
       [ 1.23423997,  1.94447577, -0.26357823, -1.28688187, -0.69355921,
        -1.10230105,  0.60470064, -0.1037951 ],
       [-0.84372629, -0.99692019, -0.16024856,  0.15569823,  0.12235685,
        -0.49346891, -0.91968415, -1.0403932 ]])

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y, stratify=y, random_state=42)

# stratify=y, ensures that the training and testing datasets have the same distribution of the target variable (y) as the original dataset. 
# We use this when teh dataset is imbalanced


In [9]:
X_train.shape

(575, 8)

In [10]:
X_test.shape

(192, 8)

In [11]:
y_train.value_counts()

Outcome
0    375
1    200
Name: count, dtype: int64

In [12]:
200/375

0.5333333333333333

In [13]:
y_test.value_counts()

Outcome
0    125
1     67
Name: count, dtype: int64

In [14]:
67/125

0.536

### **Train using a stand alone model**

In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

scores=cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores.mean()       #average of 5 -fold cross-validation scores

0.7132586367880486

### **Train using Bagging**

In [16]:
from sklearn.ensemble import BaggingClassifier
bag_model=BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),        #Specifies the type of model to use for bagging. Here, each bagged model is a decision tree.
    n_estimators=100,                               #Indicates the number of base models (decision trees in this case) to train in the ensemble.
    max_samples=0.8,                                #Specifies the proportion (80%) of the training data to sample for each base model, with replacement.
    oob_score=True,                                 #Enables the use of "out-of-bag" samples (data that was not included in training by base model) to evaluate the model’s performance.
    random_state=0
)

bag_model.fit(X_train, y_train)
bag_model.oob_score_            #Returns the accuracy score of the model calculated using the out-of-bag samples, giving you a performance metric without needing a separate validation dataset.

0.7478260869565218

In [17]:
bag_model.score(X_test, y_test)

0.7552083333333334

In [18]:
bag_model=BaggingClassifier(
    base_estimator=DecisionTreeClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0
)
scores=cross_val_score(BaggingClassifier(), X, y, cv=5)
scores

array([0.70779221, 0.69480519, 0.75816993, 0.81699346, 0.71895425])

In [19]:
scores.mean()

0.7393430099312452

#### **We can see some improvement in test score with bagging classifier as compared to a standalone classifier.**

### **Train using Random Forest**

In [20]:
from sklearn.ensemble import RandomForestClassifier
rclf_scores=cross_val_score(RandomForestClassifier(n_estimators=50, criterion='gini'), X, y, cv=5)
rclf_scores

array([0.76623377, 0.72727273, 0.74509804, 0.82352941, 0.77777778])

In [21]:
rclf_scores.mean()

0.7679823444529326

### **Using Bagging with RandomForestClassifier**

In [22]:
from sklearn.ensemble import BaggingClassifier
b_rclf=BaggingClassifier(
    base_estimator=RandomForestClassifier(n_estimators='50', criterion='gini'),
    n_estimators=10,
    max_samples=0.8,
    oob_score=True,
    random_state=42
)
b_scores=cross_val_score(BaggingClassifier(), X, y, cv=5)
b_scores

array([0.69480519, 0.70779221, 0.71895425, 0.81045752, 0.7124183 ])

In [23]:
b_scores.mean()

0.728885493591376